

# First Model

**RICE Pest Multi Classification Model. Using Muiltiple Features From the Dataset, the model can make predictions on rice to Identify If it has pests and what pest it is likely to be.**

In [ ]:
import sys, numpy as np
import pandas as pd
import os
import warnings
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from numpy import argmax

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

df= pd.read_csv('https://raw.githubusercontent.com/Russoremy/Datasets/main/RICE.csv')

df.drop(['Observation Year','Standard Week','Collection Type','Location'], axis=1, inplace=True)
print(df.columns)


Index(['Pest Value', 'MaxT', 'MinT', 'RH1(%)', 'RH2(%)', 'RF(mm)', 'WS(kmph)',
       'SSH(hrs)', 'EVP(mm)', 'PEST NAME'],
      dtype='object')


In [ ]:

X = df.drop('PEST NAME', axis=1)
y = df['PEST NAME']

In [ ]:
import numpy as np
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Split Data we are using a different data set than the one Dr. Lee used and it contains no images
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# Normalize numerical features using MinMaxScaler
numerical_features = ['Pest Value', 'MaxT', 'MinT', 'RH1(%)', 'RH2(%)', 'RF(mm)', 'WS(kmph)', 'SSH(hrs)', 'EVP(mm)']
scaler = MinMaxScaler()
X_train_numerical = scaler.fit_transform(X_train[numerical_features])
X_test_numerical = scaler.transform(X_test[numerical_features])

# Concatenate categorical and numerical features
X_train = np.concatenate([X_train_numerical], axis=1)
X_test = np.concatenate([X_test_numerical], axis=1)

# Target Encoding
encoder = LabelEncoder()

# Fit and transform the target variable
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# Reshape target variable to match the input shape
y_train = y_train_encoded.reshape(-1, 1)
y_test = y_test_encoded.reshape(-1, 1)

relu = lambda x: (x >= 0) * x
relu2deriv = lambda x: x >= 0
batch_size = 100
alpha, iterations, hidden_size, input_size = (0.001, 300, 40, X_train.shape[1])

weights_0_1 = 0.2 * np.random.random((input_size, hidden_size)) - 0.1
weights_1_2 = 0.2 * np.random.random((hidden_size, y_train.shape[1])) - 0.1

for j in range(iterations):
    error, correct_cnt = (0.0, 0)

    for i in range(len(X_train)):
        layer_0 = X_train[i:i+1]
        layer_1 = relu(np.dot(layer_0, weights_0_1))
        layer_2 = np.dot(layer_1, weights_1_2)

        error += np.sum((y_train[i:i+1] - layer_2) ** 2)
        correct_cnt += int(np.argmax(layer_2) == np.argmax(y_train[i:i+1]))

        layer_2_delta = (y_train[i:i+1] - layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

    sys.stdout.write("\r I:" + str(j) +
                     " Train-Err:" + str(error / float(len(X_train)))[0:5] +\
                     " Train-Acc:" + str(correct_cnt / float(len(X_train))))


if(j % 10 == 0 or j == iterations-1):
    error, correct_cnt = (0.0, 0)

    for i in range(len(X_test)):

        layer_0 = X_test[i:i+1]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        layer_2 = np.dot(layer_1,weights_1_2)

        error += np.sum((y_test[i:i+1] - layer_2) ** 2)
        correct_cnt += int(np.argmax(layer_2) == \
                                        np.argmax(y_test[i:i+1]))
    sys.stdout.write(" Test-Err:" + str(error/float(len(X_test)))[0:5] +\
                     " Test-Acc:" + str(correct_cnt/float(len(X_test))) + "\n")
    print()

    #The part I am stuck in and slighty confused with this type of example you showed is how I can tune it an reduce the error.
    #I also believe the keras models are more simple and efficient but this allow for more customization for tunning. Which one is better practice?


 I:299 Train-Err:10.27 Train-Acc:1.0 Test-Err:10.24 Test-Acc:1.0



# Keras Model


In [ ]:
import sys, numpy as np
import pandas as pd
import os
import warnings
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from numpy import argmax

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

df= pd.read_csv('https://raw.githubusercontent.com/Russoremy/Datasets/main/RICE.csv')

df.drop(['Observation Year','Standard Week','Collection Type','Location'], axis=1, inplace=True)
print(df.columns)

Index(['Pest Value', 'MaxT', 'MinT', 'RH1(%)', 'RH2(%)', 'RF(mm)', 'WS(kmph)',
       'SSH(hrs)', 'EVP(mm)', 'PEST NAME'],
      dtype='object')


In [ ]:
# Save the best model
cb_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='model-{epoch:02d}-{val_accuracy:.2f}.hdf5',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

# Reduce learning rate if no improvement in val_loss
cb_reducelr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    mode='min',
    factor=0.1,
    patience=10,
    verbose=1,
    min_lr=0.00001
)

# Stop training if no improvement in val_accuracy
cb_earlystop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    min_delta=0.001,
    patience=10,
    verbose=1
)

# Save training history to CSV file
cb_csvlogger = tf.keras.callbacks.CSVLogger(
    filename='training_log.csv',
    separator=',',
    append=False
)

In [ ]:
# Split Data we are using a different data set than the one Dr. Lee used and it contains no images
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# Normalize numerical features using MinMaxScaler
numerical_features = ['Pest Value', 'MaxT', 'MinT', 'RH1(%)', 'RH2(%)', 'RF(mm)', 'WS(kmph)', 'SSH(hrs)', 'EVP(mm)']
scaler = MinMaxScaler()
X_train_numerical = scaler.fit_transform(X_train[numerical_features])
X_test_numerical = scaler.transform(X_test[numerical_features])

# Concatenate categorical and numerical features
X_train = np.concatenate([X_train_numerical], axis=1)
X_test = np.concatenate([X_test_numerical], axis=1)

# Target Encoding
encoder = LabelEncoder()

# Fit and transform the target variable
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# Reshape target variable to match the input shape
y_train = y_train_encoded.reshape(-1, 1)
y_test = y_test_encoded.reshape(-1, 1)

In [ ]:
# determine the number of input features
n_features = X_train.shape[1]
n_features

9

In [ ]:
# define model
model = Sequential()
model.add(Dense(1, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(1, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='softmax'))

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# fit the model
model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=1)

Epoch 1/100
1300/1300 [==============================] - 6s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0479
Epoch 2/100
1300/1300 [==============================] - 2s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0479
Epoch 3/100
1300/1300 [==============================] - 2s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0479
Epoch 4/100
1300/1300 [==============================] - 2s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0479
Epoch 5/100
1300/1300 [==============================] - 3s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0479
Epoch 6/100
 591/1300 [============>.................] - ETA: 1s - loss: 0.0000e+00 - accuracy: 0.0467

KeyboardInterrupt: ignored

In [ ]:
# evaluate the model
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.3f' % acc)
# make a prediction
row = [1, 29.6,	11.3,	80, 32.7, 1, 0, 0, 0 ]
yhat = model.predict([row])
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

Test Accuracy: 0.050
1/1 [==============================] - 0s 85ms/step
Predicted: [[1.]] (class=0)


# Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def create_model(neurons_1=1, neurons_2=1):
    # create model
    model = Sequential()
    model.add(Dense(neurons_1, input_dim=9, kernel_initializer='uniform', activation='linear'))
    model.add(Dense(neurons_2, kernel_initializer='uniform', activation='linear'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

neurons_1 = [1, 5, 10, 15, 20, 25, 30, 50, 100]
neurons_2 = [1, 5, 10, 15, 20, 25, 30, 50, 100]
param_grid = dict(neurons_1=neurons_1, neurons_2=neurons_2)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

grid_result = grid.fit(X_train, y_train, callbacks=[cb_earlystop], validation_split=0.2)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Epoch 11: early stopping
Best: 0.047923 using {'neurons_1': 1, 'neurons_2': 1}
